In [14]:
# Welcome to your new notebook
# Type here in the cell editor to add code!


StatementMeta(, ebf730cf-a2c7-476b-9d81-3ac4bb4949d0, 16, Finished, Available, Finished)

In [15]:
df = spark.read.option("multiline", "true").json("Files/bing-latest-news.json")
# df now is a Spark DataFrame containing JSON data from "Files/bing-latest-news.json".
display(df)

StatementMeta(, ebf730cf-a2c7-476b-9d81-3ac4bb4949d0, 17, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 3010e3b7-1bb1-4261-92f3-5efd80d1d4a8)

In [16]:
from pyspark.sql.functions import explode
df_exploded = df.select(explode(df["value"]).alias("json_object"))

StatementMeta(, ebf730cf-a2c7-476b-9d81-3ac4bb4949d0, 18, Finished, Available, Finished)

In [17]:
display(df_exploded)

StatementMeta(, ebf730cf-a2c7-476b-9d81-3ac4bb4949d0, 19, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a59b7623-6148-45f1-a687-ddeb9fe9477f)

In [18]:
json_list = df_exploded.toJSON().collect()

StatementMeta(, ebf730cf-a2c7-476b-9d81-3ac4bb4949d0, 20, Finished, Available, Finished)

## Testing the JSON string list

In [19]:
print(json_list[9])

StatementMeta(, ebf730cf-a2c7-476b-9d81-3ac4bb4949d0, 21, Finished, Available, Finished)

{"json_object":{"about":[{"name":"Gulf of Mexico","readLink":"https://api.bing.microsoft.com/api/v7/entities/4225f1fe-45d3-b433-17c0-0d39c284fe91"},{"name":"Havana","readLink":"https://api.bing.microsoft.com/api/v7/entities/3ca5066a-a331-c1b3-135f-1fde36da55db"}],"category":"World","datePublished":"2024-11-07T14:33:00.0000000Z","description":"Hurricane Rafael continued to lose steam in the Gulf of Mexico on Thursday after striking Cuba as a Category 3 major hurricane one day earlier. As of 7 a.m., Rafael was still a Category 2","image":{"thumbnail":{"contentUrl":"https://www.bing.com/th?id=OVFT.01E7vKqYIUqZ5eO42Ht8hy&pid=News","height":436,"width":700}},"name":"Hurricane Rafael heads into Gulf of Mexico after striking Cuba as major hurricane","provider":[{"_type":"Organization","image":{"thumbnail":{"contentUrl":"https://www.bing.com/th?id=ODF.DOagdQ9zAAn-8mg5GBFfNQ&pid=news"}},"name":"Orlando Sentinel"}],"url":"https://www.orlandosentinel.com/2024/11/06/system-that-could-grow-into-hur

In [20]:
import json
news_json = json.loads(json_list[9])

StatementMeta(, ebf730cf-a2c7-476b-9d81-3ac4bb4949d0, 22, Finished, Available, Finished)

In [21]:
print(news_json['json_object']['name'])
print(news_json['json_object']['description'])
print(news_json['json_object']['category'])
print(news_json['json_object']['url'])
print(news_json['json_object']['image']['thumbnail']['contentUrl'])
print(news_json['json_object']['provider'][0]['name'])
print(news_json['json_object']['datePublished'])

StatementMeta(, ebf730cf-a2c7-476b-9d81-3ac4bb4949d0, 23, Finished, Available, Finished)

Hurricane Rafael heads into Gulf of Mexico after striking Cuba as major hurricane
Hurricane Rafael continued to lose steam in the Gulf of Mexico on Thursday after striking Cuba as a Category 3 major hurricane one day earlier. As of 7 a.m., Rafael was still a Category 2
World
https://www.orlandosentinel.com/2024/11/06/system-that-could-grow-into-hurricane-rafael-headed-toward-gulf-of-mexico-nhc-says/
https://www.bing.com/th?id=OVFT.01E7vKqYIUqZ5eO42Ht8hy&pid=News
Orlando Sentinel
2024-11-07T14:33:00.0000000Z


## Processing the JSON Property List

In [22]:
title = []
description = []
category = []
image = []
url = []
provider = []
datePublished = []

#Process each JSON object in the list
for json_str in json_list:
    try:
        #Parse the JSON string into a dictionary
        article = json.loads(json_str)

        if article['json_object'].get('category') and article['json_object'].get('image', {}).get('thumbnail', {}).get('contentUrl'):

   
            #Extract information from the disctionary
            title.append(article['json_object']['name'])
            description.append(article['json_object']['description'])
            category.append(article['json_object']['category'])
            url.append(article['json_object']['url'])
            image.append(article['json_object']['image']['thumbnail']['contentUrl'])
            provider.append(article['json_object']['provider'][0]['name'])
            datePublished.append(article['json_object']['datePublished'])

    except Exception as e:
            print(f"Error processing JSON object: {e}")

StatementMeta(, ebf730cf-a2c7-476b-9d81-3ac4bb4949d0, 24, Finished, Available, Finished)

## Converting the List to a Dataframe

In [23]:
from pyspark.sql.types import StructType, StructField, StringType


#Combine the Lists
data = list(zip(title,description,category,url,image,provider,datePublished))

# Define Schema
schema = StructType([
    StructField('title', StringType(), True),
    StructField('description', StringType(), True),
    StructField('category', StringType(), True),
    StructField('url', StringType(), True),
    StructField('image', StringType(), True),
    StructField('provider', StringType(), True),
    StructField('datePublished', StringType(), True)
])

# Create Dataframe
df_cleaned = spark.createDataFrame(data, schema=schema)

StatementMeta(, ebf730cf-a2c7-476b-9d81-3ac4bb4949d0, 25, Finished, Available, Finished)

In [24]:
display(df_cleaned.limit(10))

StatementMeta(, ebf730cf-a2c7-476b-9d81-3ac4bb4949d0, 26, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 09630073-4920-4923-b38d-f15adfd61290)

## Processing the Date Column

In [25]:
from pyspark.sql.functions import to_date, date_format

df_cleaned_final = df_cleaned.withColumn('datePublished',date_format(to_date('datePublished'),'dd-MMM-yyyy'))

StatementMeta(, ebf730cf-a2c7-476b-9d81-3ac4bb4949d0, 27, Finished, Available, Finished)

In [26]:
display(df_cleaned_final)

StatementMeta(, ebf730cf-a2c7-476b-9d81-3ac4bb4949d0, 28, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 0545dffa-a1de-48ca-8e36-4ab05d8aca2a)

## Writing the Final Dataframe to the Lakehouse DB in a Delta Format

#### Implementing Incremental Load

In [27]:
from pyspark.sql.utils import AnalysisException

try: 
    
    table_name = 'bing_lake_db.tbl_latest_news'

    df_cleaned_final.write.format('delta').saveAsTable(table_name)

except AnalysisException:

    print('Table Already Exists')

    df_cleaned_final.createOrReplaceTempView('vw_df_cleaned_final')


    spark.sql(f""" MERGE INTO {table_name} target_table
                   USING vw_df_cleaned_final source_view
                  
                   ON source_view.url = target_table.url

                   WHEN MATCHED and
                   source_view.title <> target_table.title OR
                   source_view.description <> target_table.description OR
                   source_view.category <> target_table.category OR
                   source_view.image <> target_table.image OR
                   source_view.provider <> target_table.provider OR
                   source_view.datePublished <> target_table.datePublished

                   THEN UPDATE SET *

                   WHEN NOT MATCHED THEN INSERT *
                   
                """)


StatementMeta(, ebf730cf-a2c7-476b-9d81-3ac4bb4949d0, 29, Finished, Available, Finished)

Table Already Exists


In [28]:
%%sql

SELECT count(*) from bing_lake_db.tbl_latest_news

StatementMeta(, ebf730cf-a2c7-476b-9d81-3ac4bb4949d0, 30, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 1 fields>